In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib import style
%matplotlib inline  
import seaborn as sns
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize


In [2]:
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
stop_words=set(nltk.corpus.stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
from nltk.corpus import stopwords 
from nltk import word_tokenize,sent_tokenize 
from nltk.stem import PorterStemmer,LancasterStemmer 
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  
from nltk import ne_chunk
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [9]:
from google.colab import files
uploaded = files.upload()

Saving Reviews.csv to Reviews.csv


In [4]:
df = pd.read_csv('Reviews.csv')

FileNotFoundError: ignored

In [11]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [12]:
print(df.shape)
print(df.isnull().values.any())

(568454, 10)
True


In [40]:
df.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator','Score','Time','Summary'],axis=1,inplace=True)

In [14]:
df.head()

,Text
0,I have bought several of the Vitality canned d...
1,Product arrived labeled as Jumbo Salted Peanut...
2,This is a confection that has been around a fe...
3,If you are looking for the secret ingredient i...
4,Great taffy at a great price. There was a wid...


In [41]:
def cleanText(headline):
  le=WordNetLemmatizer()
  wordTokens=word_tokenize(headline)
  tokens=[le.lemmatize(w) for w in wordTokens if w not in stop_words and len(w)>3]
  cleanedText=" ".join(tokens)
  return cleanedText

In [42]:
df['Text']=df['Text'].apply(cleanText)

In [17]:
vectors =TfidfVectorizer(stop_words=stop_words,max_features=1000,ngram_range = (1,3))
vectText=vectors.fit_transform(df['Text'])

In [18]:
print(vectText.shape)
print(vectText)

(568454, 1000)
  (0, 504)	0.30318304462156975
  (0, 115)	0.3162363190689679
  (0, 72)	0.3351026921211797
  (0, 799)	0.21821772496059946
  (0, 535)	0.2568130207133922
  (0, 681)	0.3184701536414067
  (0, 483)	0.11156894352481986
  (0, 503)	0.21921563658053317
  (0, 691)	0.20403716683578527
  (0, 365)	0.11967955811066665
  (0, 336)	0.18080901287346596
  (0, 682)	0.38084596642628005
  (0, 331)	0.16046827628977436
  (0, 114)	0.27422386993031245
  (0, 768)	0.22828365250331556
  (0, 90)	0.1831131220264846
  (1, 862)	0.3226831290792325
  (1, 792)	0.48082653289402927
  (1, 797)	0.32175021423903283
  (1, 10)	0.3368554946210655
  (1, 633)	0.39207415353666536
  (1, 45)	0.36999810052437265
  (1, 682)	0.39631538273241423
  (2, 409)	0.22961784148439382
  (2, 918)	0.36282717813408827
  :	:
  (568451, 25)	0.12391228067885442
  (568451, 325)	0.1013909254411724
  (568451, 518)	0.14536263185925224
  (568451, 442)	0.3179036192854717
  (568451, 918)	0.1498203400915397
  (568451, 797)	0.1608312782083553
  (5

In [19]:
from sklearn.decomposition import TruncatedSVD
lsa = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10, random_state=42)
lsaTop=lsa.fit_transform(vectText)

In [20]:
print(lsaTop)
print(lsaTop.shape)

[[ 2.44582460e-01 -1.23411914e-01  1.28098365e-01 ...  1.38027926e-01
   8.42362238e-02  1.53210359e-02]
 [ 1.36218396e-01 -7.94573968e-02  1.75130919e-02 ...  1.23242744e-01
   1.22555607e-01 -7.97698397e-02]
 [ 1.10956440e-01 -6.03839765e-02  3.82574440e-02 ... -1.63681793e-02
   3.66673064e-02 -5.58644608e-05]
 ...
 [ 2.59782369e-01 -1.62093827e-01  2.02888010e-01 ...  2.70714350e-03
  -2.96087192e-02 -1.46170860e-02]
 [ 1.51842314e-01 -7.66606248e-02  3.06543820e-02 ... -8.47479104e-02
   2.77411378e-03  1.34980178e-01]
 [ 7.19389561e-02 -4.64676887e-02 -6.09042003e-03 ...  7.85356648e-02
   4.91431831e-02 -2.64260840e-02]]
(568454, 10)


In [21]:
l=lsaTop[0]
print("First Document :")
for i,topics in enumerate(l):
  print("Topic ",i," : ",topics*100)

First Document :
Topic  0  :  24.45824597624286
Topic  1  :  -12.341191431524317
Topic  2  :  12.809836461838161
Topic  3  :  3.0659532780099785
Topic  4  :  13.055016376057493
Topic  5  :  -10.567633778992949
Topic  6  :  -9.466065453527841
Topic  7  :  13.802792624079615
Topic  8  :  8.423622377662813
Topic  9  :  1.5321035944064163


In [22]:
vocab = vectors.get_feature_names_out()

for i, comp in enumerate(lsa.components_):
    vocabComp = zip(vocab, comp)
    sorted_words = sorted(vocabComp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for s in sorted_words:
        print(s[0],end=" ")
    print("\n")

Topic 0: 
coffee like taste good flavor great product love food would 

Topic 1: 
coffee strong cup roast bold blend keurig bitter cups drink 

Topic 2: 
food coffee treat dog cat love amazon product year price 

Topic 3: 
product amazon com amazon com www www amazon www amazon com http http www http www amazon 

Topic 4: 
food like taste www http http www www amazon com www amazon http www amazon amazon com gp 

Topic 5: 
treat love dog like www www amazon www amazon com http http www http www amazon 

Topic 6: 
great food love chip flavor snack chocolate free gluten best 

Topic 7: 
great product taste great product taste great water food drink great taste recommend 

Topic 8: 
chocolate product free cooky gluten gluten free good coffee bar peanut 

Topic 9: 
good chip price taste taste good really treat snack really good good price 



In [23]:
Topics = []
for i, comp in enumerate(lsa.components_):
    vocabComp = zip(vocab, comp)
    st=''
    sorted_words = sorted(vocabComp, key= lambda x:x[1], reverse=True)[:10]
    for t in sorted_words:
        st = st + t[0] + " "
    Topics.append(st)
Topics

['coffee like taste good flavor great product love food would ',
 'coffee strong cup roast bold blend keurig bitter cups drink ',
 'food coffee treat dog cat love amazon product year price ',
 'product amazon com amazon com www www amazon www amazon com http http www http www amazon ',
 'food like taste www http http www www amazon com www amazon http www amazon amazon com gp ',
 'treat love dog like www www amazon www amazon com http http www http www amazon ',
 'great food love chip flavor snack chocolate free gluten best ',
 'great product taste great product taste great water food drink great taste recommend ',
 'chocolate product free cooky gluten gluten free good coffee bar peanut ',
 'good chip price taste taste good really treat snack really good good price ']